# MODIS Data Analysis for Saskatchewan Glacier

Interactive analysis of MCD10A1 (snow cover) and MCD43A3 (albedo) data for glacier research.


## 1. Setup and Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pathlib import Path

# MODIS tools
from modis_tools.auth import ModisSession
from modis_tools.resources import CollectionApi, GranuleApi
from modis_tools.granule_handler import GranuleHandler

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("✅ All imports successful!")

## 2. Authentication and Session Setup

In [ ]:
# Create session (uses .netrc file if available)
session = ModisSession()
print(f"✅ Session created: {type(session).__name__}")
print(f"Session object: {session}")

## 3. Debug Collection Queries

In [ ]:
# Create collection client
collection_client = CollectionApi(session=session)
print(f"✅ Collection client created: {type(collection_client).__name__}")

In [ ]:
# Test general MODIS search first
print("🔍 Testing general MODIS search...")
try:
    modis_collections = collection_client.query(keyword="MODIS")
    modis_list = list(modis_collections)
    print(f"✅ Found {len(modis_list)} MODIS collections total")
    
    # Show first few
    print("\nFirst 10 MODIS collections:")
    for i, col in enumerate(modis_list[:10]):
        print(f"  {i+1:2d}. {getattr(col, 'short_name', 'Unknown')} v{getattr(col, 'version', 'Unknown')}")
        
except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
# Test MCD10A1 specifically
print("🔍 Testing MCD10A1 queries...")

test_queries = [
    {"short_name": "MCD10A1", "version": "061"},
    {"short_name": "MCD10A1", "version": "6"},
    {"short_name": "MCD10A1"},
    {"keyword": "MCD10A1"},
    {"keyword": "snow cover"},
]

mcd10a1_found = None

for i, query in enumerate(test_queries, 1):
    try:
        print(f"\nQuery {i}: {query}")
        collections = collection_client.query(**query)
        collections_list = list(collections)
        print(f"   ✅ Found {len(collections_list)} collections")
        
        if collections_list:
            for j, collection in enumerate(collections_list[:3]):
                title = getattr(collection, 'title', 'N/A')
                short_name = getattr(collection, 'short_name', 'N/A')
                version = getattr(collection, 'version', 'N/A')
                concept_id = getattr(collection, 'concept_id', 'N/A')
                
                print(f"      Collection {j+1}:")
                print(f"        Title: {title}")
                print(f"        Short Name: {short_name}")
                print(f"        Version: {version}")
                print(f"        Concept ID: {concept_id}")
                
                # Save the first MCD10A1 collection found
                if short_name == "MCD10A1" and mcd10a1_found is None:
                    mcd10a1_found = collection
                    print(f"        ⭐ Saved as MCD10A1 collection!")
                    
    except Exception as e:
        print(f"   ❌ Error: {e}")

print(f"\n🎯 MCD10A1 Collection found: {mcd10a1_found is not None}")

In [ ]:
# Test MCD43A3 specifically
print("🔍 Testing MCD43A3 queries...")

test_queries_43a3 = [
    {"short_name": "MCD43A3", "version": "061"},
    {"short_name": "MCD43A3", "version": "6"},
    {"short_name": "MCD43A3"},
    {"keyword": "MCD43A3"},
    {"keyword": "albedo"},
]

mcd43a3_found = None

for i, query in enumerate(test_queries_43a3, 1):
    try:
        print(f"\nQuery {i}: {query}")
        collections = collection_client.query(**query)
        collections_list = list(collections)
        print(f"   ✅ Found {len(collections_list)} collections")
        
        if collections_list:
            for j, collection in enumerate(collections_list[:3]):
                title = getattr(collection, 'title', 'N/A')
                short_name = getattr(collection, 'short_name', 'N/A')
                version = getattr(collection, 'version', 'N/A')
                concept_id = getattr(collection, 'concept_id', 'N/A')
                
                print(f"      Collection {j+1}:")
                print(f"        Title: {title}")
                print(f"        Short Name: {short_name}")
                print(f"        Version: {version}")
                print(f"        Concept ID: {concept_id}")
                
                # Save the first MCD43A3 collection found
                if short_name == "MCD43A3" and mcd43a3_found is None:
                    mcd43a3_found = collection
                    print(f"        ⭐ Saved as MCD43A3 collection!")
                    
    except Exception as e:
        print(f"   ❌ Error: {e}")

print(f"\n🎯 MCD43A3 Collection found: {mcd43a3_found is not None}")

## 4. Query Granules (if collections found)

In [ ]:
# Saskatchewan Glacier bounding box
saskatchewan_bbox = [-117.3, 52.1, -117.1, 52.3]  # [west, south, east, north]

# Test date range
start_date = "2023-07-01"
end_date = "2023-07-31"

print(f"📍 Bounding box: {saskatchewan_bbox}")
print(f"📅 Date range: {start_date} to {end_date}")

In [ ]:
# Test granule query for MCD10A1 if collection was found
if mcd10a1_found:
    print("🔍 Testing MCD10A1 granule query...")
    try:
        granule_client = GranuleApi.from_collection(mcd10a1_found, session=session)
        granules = granule_client.query(
            start_date=start_date,
            end_date=end_date,
            bounding_box=saskatchewan_bbox,
            limit=5  # Limit for testing
        )
        
        granules_list = list(granules)
        print(f"✅ Found {len(granules_list)} MCD10A1 granules")
        
        # Show details of first granule
        if granules_list:
            granule = granules_list[0]
            print(f"\nFirst granule details:")
            for attr in dir(granule):
                if not attr.startswith('_'):
                    try:
                        value = getattr(granule, attr)
                        if not callable(value):
                            print(f"  {attr}: {value}")
                    except:
                        pass
        
    except Exception as e:
        print(f"❌ Error querying MCD10A1 granules: {e}")
else:
    print("❌ Cannot test granules - no MCD10A1 collection found")

In [ ]:
# Test granule query for MCD43A3 if collection was found
if mcd43a3_found:
    print("🔍 Testing MCD43A3 granule query...")
    try:
        granule_client = GranuleApi.from_collection(mcd43a3_found, session=session)
        granules = granule_client.query(
            start_date=start_date,
            end_date=end_date,
            bounding_box=saskatchewan_bbox,
            limit=5  # Limit for testing
        )
        
        granules_list = list(granules)
        print(f"✅ Found {len(granules_list)} MCD43A3 granules")
        
        # Show details of first granule
        if granules_list:
            granule = granules_list[0]
            print(f"\nFirst granule details:")
            for attr in dir(granule):
                if not attr.startswith('_'):
                    try:
                        value = getattr(granule, attr)
                        if not callable(value):
                            print(f"  {attr}: {value}")
                    except:
                        pass
        
    except Exception as e:
        print(f"❌ Error querying MCD43A3 granules: {e}")
else:
    print("❌ Cannot test granules - no MCD43A3 collection found")

## 5. Download Test (Small Sample)

In [ ]:
# Create data directories
data_dir = Path("./")
snow_dir = data_dir / "MCD10A1_snow_cover"
albedo_dir = data_dir / "MCD43A3_albedo"

snow_dir.mkdir(exist_ok=True)
albedo_dir.mkdir(exist_ok=True)

print(f"📁 Created directories:")
print(f"   Snow: {snow_dir}")
print(f"   Albedo: {albedo_dir}")

In [ ]:
# Test download of 1-2 files if granules were found
downloaded_files = []

# Download MCD10A1 test file
if 'granules_list' in locals() and mcd10a1_found:
    print("📥 Testing MCD10A1 download...")
    try:
        # Re-query with limit 1 for download test
        granule_client = GranuleApi.from_collection(mcd10a1_found, session=session)
        test_granules = granule_client.query(
            start_date=start_date,
            end_date=end_date,
            bounding_box=saskatchewan_bbox,
            limit=1
        )
        
        test_granules_list = list(test_granules)
        if test_granules_list:
            files = GranuleHandler.download_from_granules(
                test_granules_list, 
                session=session, 
                path=str(snow_dir)
            )
            downloaded_files.extend(files)
            print(f"✅ Downloaded {len(files)} MCD10A1 file(s)")
        
    except Exception as e:
        print(f"❌ Download error: {e}")

# Download MCD43A3 test file
if 'granules_list' in locals() and mcd43a3_found:
    print("📥 Testing MCD43A3 download...")
    try:
        # Re-query with limit 1 for download test
        granule_client = GranuleApi.from_collection(mcd43a3_found, session=session)
        test_granules = granule_client.query(
            start_date=start_date,
            end_date=end_date,
            bounding_box=saskatchewan_bbox,
            limit=1
        )
        
        test_granules_list = list(test_granules)
        if test_granules_list:
            files = GranuleHandler.download_from_granules(
                test_granules_list, 
                session=session, 
                path=str(albedo_dir)
            )
            downloaded_files.extend(files)
            print(f"✅ Downloaded {len(files)} MCD43A3 file(s)")
        
    except Exception as e:
        print(f"❌ Download error: {e}")

print(f"\n📊 Total files downloaded: {len(downloaded_files)}")
for f in downloaded_files:
    print(f"   {f}")

## 6. Summary and Next Steps

In [ ]:
print("📋 DIAGNOSTIC SUMMARY")
print("=" * 50)
print(f"✅ Authentication successful: {session is not None}")
print(f"✅ Collection client created: {collection_client is not None}")
print(f"🎯 MCD10A1 collection found: {mcd10a1_found is not None}")
print(f"🎯 MCD43A3 collection found: {mcd43a3_found is not None}")
print(f"📥 Files downloaded: {len(downloaded_files)}")

print("\n🔧 TROUBLESHOOTING TIPS:")
if mcd10a1_found is None and mcd43a3_found is None:
    print("❌ No collections found - possible issues:")
    print("   1. Check NASA Earthdata credentials")
    print("   2. Verify account has MODIS data access")
    print("   3. Check server connectivity")
    print("   4. Try different product names or versions")
elif len(downloaded_files) == 0:
    print("⚠️  Collections found but no downloads - possible issues:")
    print("   1. Check date range (summer months recommended)")
    print("   2. Verify bounding box covers data availability")
    print("   3. Check disk space and permissions")
else:
    print("✅ Everything working! Ready for full analysis.")
    print("   Next steps:")
    print("   1. Increase date range for more data")
    print("   2. Process downloaded files")
    print("   3. Create time series analysis")